# Resource Registration BB - Harvester Demo

[Flowable](https://www.flowable.com/open-source/docs/) is an Open Source workflow and Business Process Management (BPM) platform. It has a [REST API](https://registration-harvester-api.develop.eoepca.org/flowable-rest/docs) to manage and execute business processes:
- deploy BPMN 2.0 process definitions
- creating process instances of those process definitions
- accessing active or historical process instances and related data
- etc.
  
In this demo we will use the REST API to deploy and start workflows.

### Workflow deployment

In [ ]:
from requests import Session
from requests.auth import HTTPBasicAuth
import json

# Setup connection to Flowable API
flowable_base_url = "https://registration-harvester-api.develop.eoepca.org/flowable-rest"
flowable_rest_user = "eoepca"
flowable_rest_pw = "eoepca"
requests_ca_bundle = None
flowable_session = Session()
flowable_session.auth = HTTPBasicAuth(flowable_rest_user, flowable_rest_pw)
if requests_ca_bundle is not None:
    flowable_session.verify = requests_ca_bundle

# List deployed workflows (processes)
response = flowable_session.get(url=f"{flowable_base_url}/service/repository/process-definitions")
processes = response.json()["data"]
for idx, process in enumerate(processes, 1):
    print(f"{idx} {process['name']} ({process['id']})")

### Execute Landsat workflow

The Landsat harvesting workflow consists of two BPMN processes. The main process (Landsat Registration) searches for new data at USGS. For each new scene found, the workflow executes another process (Landsat Scene Ingestion) which performs the individual steps for harvesting and registering the data.

![Landsat workflow](img/landsat-workflow-bpmn.png)

In [ ]:
# Get process id for Landsat workflow
for process in processes:
    if process["name"] == "Landsat Workflow":
        processId = process["id"]    

# Define the filter for a manual workflow execution
query = json.dumps({ "created": { "gte": "2024-12-13T15:00:00.000000Z", "lt": "2024-12-13T16:00:00.000000Z" } })
variables = [
    # {"name": "collections", "type": "string", "value": "landsat-c2l2-sr,landsat-c2l1"},
    {"name": "collections", "type": "string", "value": "landsat-c2l2-sr"},
    {"name": "bbox", "type": "string", "value": "8,40,18,60"},
    {"name": "query", "type": "string", "value": query},
]

# Create the JSON body for HTTP request which triggers the workflow
body = {}
body["processDefinitionId"] = processId
body["variables"] = variables
print(json.dumps(body, indent=4))

In [ ]:
# Send request to start the workflow
response = flowable_session.post(url=f"{flowable_base_url}/service/runtime/process-instances", json=body)
print(response.status_code)
#print(json.dumps(response.json(), indent=4))
print(f"Created process instance at {response.json()["url"]}")

### Execute Sentinel workflow

The Sentinel harvesting workflow consists of two BPMN processes. The main process (Sentinel Registration Hourly) will be executed automatically be the Flowable engine every hour and searches for new data at CDSE. For each new scene discovered, the workflow  executes another process (Sentinel Scene Ingestion) which performs the individual steps for harvesting and registering the data.

![Sentinel workflow](img/sentinel-workflow-bpmn.png)

In [ ]:
# Get process id for Sentinel workflow
for process in processes:
    if process["name"] == "Sentinel Registration":
        processId = process["id"]

# Define the filter for a manual workflow execution
start_time = "2025-01-06T00:00:00.000000Z"
stop_time = "2025-01-08T00:00:00.000000Z"
datetime = f"ContentDate/Start ge {start_time} and ContentDate/Start lt {stop_time}"
collection = "startswith(Name,'S2') and contains(Name,'L2A') and not contains(Name,'_N9999')"
spatial = "intersects(area=geography'SRID=4326;POLYGON((3 55, 3 47, 18 47, 18 55, 3 55))')"
online = "Online eq true"
cloudcover = "Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10)"
odata_filter = f"({datetime}) and ({collection}) and ({spatial}) and ({online}) and ({cloudcover})"

# Create the JSON body for HTTP request which triggers the workflow
body = {}
body["processDefinitionId"] = processId
body["variables"] = [
    {
        "name": "filter",
        "type": "string",
        "value": odata_filter,
    }
]
print(json.dumps(body, indent=4))

In [ ]:
# Send request to start the workflow
response = flowable_session.post(url=f"{flowable_base_url}/service/runtime/process-instances", json=body)
print(response.status_code)
#print(json.dumps(response.json(), indent=4))
print(f"Created process instance at {response.json()["url"]}")